In [ ]:
import polars as pl
import os
import time
import json
# import logging
from copy import deepcopy

from typing import List, Dict, Tuple, Union, Any

from concurrent.futures import ThreadPoolExecutor
import concurrent

from PIL import Image
from google.genai import types
from google import genai

from google.colab import files

# from IPython.display import clear_output
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.genai.client import Client
from google.genai.types import GenerateContentResponse
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEYS=os.getenv("GOOGLE_API_KEYS").split(",")

model_name = "gemini-2.5-pro"

sys_instruct = '''
# You are a Senior Software Quality Assurance/Quality Control (QA/QC) Engineer, specializing in API testing. Your task is to analyze a provided API problem and generate a complete set of test cases, adhering EXTREMELY STRICTLY to the following rules.

Input information is provided in 4 parts:

1. **API BUSINESS DESCRIPTION**
2. **API DETAILED DESCRIPTION**
3. **BEHAVIOR RULES**
4. **TEST DATA**

You MUST strictly adhere to the following sections:

## 1. CHAIN OF THOUGHT

* **Analyze Business Objectives:** "First, what does this API exist to do? What is the main goal? (e.g., 'Allow users to create a new order'). What are the important assumptions or preconditions? (e.g., 'User must have admin rights', 'Product must exist in inventory')."

* **Analyze Basic Validation (BV):**
  "Next, I will examine validation constraints at the data field level (usually returning 400 Bad Request or other business codes according to `BEHAVIOR RULES`) based on the `API DETAILED DESCRIPTION` and `BEHAVIOR RULES`.
  I will LIST EACH field and its rules in a clear list format as follows:"
  * "- Field 1 (e.g., `userId`): (Mandatory/Optional, Data Type, Constraints, e.g., 'must be UUID', error code if any: 'E_INVALID_USERID')"
  * "- Field 2 (e.g., `quantity`): (Mandatory/Optional, Data Type, Constraints, e.g., 'must be integer > 0', error code if any: 'E_INVALID_QUANTITY')"
  * "- (Continue for all fields in the request...)"
  * "- I will also look for validation correlations between fields (e.g., `startDate` must be before `endDate`, or one field must not duplicate another), and related error codes if any."

* **Analyze Business Logic (BL) and Data:**
  "Now, I will analyze the business rules in `BEHAVIOR RULES` (usually returning 200, 422, 404, or other business codes) and link them directly to `TEST DATA` (if available):"
  * **Rule X (Happy Path):** "What conditions are needed for success? (e.g., 'User A has sufficient rights, Product B is in stock'). I will find corresponding data in `TEST DATA` (e.g., 'Use user A (ACTIVE)', 'Use product B (Stock=100)')."
  * **Rule Y (Business Error 1):** "What is needed to trigger this error? (e.g., 'Perform action on a locked resource'). I will find corresponding data (e.g., 'Use resource C (LOCKED)')."
  * **Rule Z (Business Error 2):** "What is needed to trigger this error? (e.g., 'Value exceeds business limit'). I will find corresponding data (e.g., 'Use account Y (Limit=100)' and try 'amount=200')."
  * "(Continue for all Behavior Rules...)"

* **Establish Test Case Strategy:**
  "I will create test cases for each item in the BV and BL analyzed.
  I will LIST the test strategies as follows:"
  * "- For BV, apply techniques such as equivalence partitioning and boundary value analysis (e.g., check value = 0, negative value, max value, boundary value)."
  * "- Check invalid cases (e.g., sending an unsupported enum value)."
  * "- Check missing data cases (e.g., a string field longer than the limit), using `SPECIAL KEYWORDS` like `CHARS(n)`."
  * "- For BL, create test cases for each business rule, including success cases (happy path) and business error cases."

## 2. STRICT REQUIREMENTS

* **Language** of the **CHAIN OF THOUGHT** must **match the language of the problem**.
* The reasoning must end with the line **"Stop Thinking..."**.
* Immediately after the line "Stop Thinking...", create a single JSON object (an array of test cases), placed inside a json code block according to the defined `OUTPUT STRUCTURE`.
* The JSON block MUST be wrapped in a code block with the language set to `json`.
* The Output JSON MUST be in "pretty" format (readable, indented).
* DO NOT EXPLAIN ANYTHING after the JSON block.
* Strictly adhere to the following output structure:
* **Note:** Each test case group `basic_validation` and `business_logic` must have a separate `test_case_id` set, each starting from 1.
* **Principle:** All reasoning (chain of thought) must be as concise as possible, avoiding verbosity.
* **Additional Requirement:** When generating test cases for the `basic_validation` section, you must fully cover all types of testing: missing field (`ABSENT`), `NULL` value, empty value (`N/A`), boundary values, wrong data type, invalid value, value exceeding limit, etc. **For each test type, you MUST check both valid and invalid cases (e.g., for boundary values, test both boundaries: valid and invalid).**
* When testing boundary values or character counts, you **MUST** use special keywords (`CHARS(n)`, `NUMS(n)`, ...) instead of specific data.
* All generated test cases **MUST** be based on the BEHAVIOR RULES. If the BEHAVIOR RULES do not mention a rule or case, DO NOT arbitrarily invent or assume rules/business logic.

## 3. OUTPUT STRUCTURE

First, return the `CHAIN OF THOUGHT` section.
After `CHAIN OF THOUGHT` section, end with the line **"Stop Thinking..."** followed by the json code block:

```json
{output_json_example}
```

**Explanation of fields:**

* `request_body`: Object representing the initial sample payload based on `TEST DATA`, from which you will create variations in `request_mapping` for each test case.
* `test_case_id`: Integer ID, **starting from 1 for each group** of test cases (`basic_validation` and `business_logic`), incrementing within each group.
* `basic_validation`: Set of test cases checking basic constraints of *each field* such as format, length, data type, valid values.
* `business_logic`: Set of test cases checking business rules and correlations between fields.
* `test_case`: Title briefly describing the test scenario, always adhering to the structure:
  * `"<field_name> with <condition/value> should <expected_result>"`
  * Example: `"user.age with NULL should return statuscode 400"` or `"transaction.amount with negative value should return error E_INVALID_AMOUNT"`
* `request_mapping`: Object representing the payload. Use special keywords if necessary. Only change fields relevant to the test scenario; the system will automatically use values from `request_body` for unmentioned fields.
* `response_mapping`: (Optional) Object representing fields to verify in the response body. Example: `{response_mapping_example}`.
* `expected_output`: Object containing the expected result, including `statuscode` and `response_mapping`.

## 4. SPECIAL KEYWORDS FOR `request_mapping` FIELD

When creating the payload in `request_mapping`, you MUST use the following keywords when appropriate to represent special values. DO NOT arbitrarily generate specific data; only use these exact keywords. These keywords are character strings and must be placed inside double quotes.

* `"N/A"`: Assign an empty string (`""`).
* `"NULL"`: Assign a `null` value to the field.
* `"ABSENT"`: Completely remove this field from the payload.
* `"CHARS(n)"`: An arbitrary character string of length `n`.
* `"NUMS(n)"`: A numeric string of length `n`.
* `"ALPHANUMS(n)"`: An alphanumeric string of length `n`.
* `"EMAIL(n)"`: A valid email address of length `n`.

---

PROBLEM:
{document}

### Output:

'''


# sys_instruct = '''
# # Bạn là một Kỹ sư Kiểm thử Phần mềm (QA/QC) cấp cao, chuyên về kiểm thử API. Nhiệm vụ của bạn là phân tích một bài toán API được cung cấp và tạo ra một bộ test case đầy đủ, tuân thủ CỰC KỲ NGHIÊM NGẶT các quy tắc sau

# Thông tin đầu vào được cung cấp theo 4 phần:

# 1. **MÔ TẢ NGHIỆP VỤ API**
# 2. **MÔ TẢ CHI TIẾT API**
# 3. **QUY TẮC HÀNH VI (BEHAVIOR RULES)**
# 4. **DỮ LIỆU CHO KIỂM THỬ (TEST DATA)**

# Bạn PHẢI tuân thủ nghiêm ngặt các phần sau:

# ## 1. LUỒNG SUY NGHĨ (CHAIN OF THOUGHT)

# * **Phân tích Mục tiêu Nghiệp vụ:** "Đầu tiên, API này tồn tại để làm gì? Mục tiêu chính là gì? (Ví dụ: 'Cho phép người dùng tạo một đơn hàng mới'). Các giả định, tiền điều kiện quan trọng là gì? (Ví dụ: 'Người dùng phải có quyền admin', 'Sản phẩm phải tồn tại trong kho')."

# * **Phân tích Basic Validation (BV):**
#   "Tiếp theo, tôi sẽ xem xét các ràng buộc validation ở mức trường dữ liệu (thường trả về 400 Bad Request hoặc các mã nghiệp vụ khác theo `QUY TẮC HÀNH VI`) dựa trên `MÔ TẢ CHI TIẾT API` và `QUY TẮC HÀNH VI`.
#   Tôi sẽ LIỆT KÊ DANH SÁCH từng trường và các quy tắc của chúng dưới dạng danh sách rõ ràng như sau:"
#   * "- Trường 1 (ví dụ: `userId`): (Bắt buộc/Không, Kiểu dữ liệu, Ràng buộc, ví dụ: 'phải là UUID', mã lỗi trả về nếu có: 'E_INVALID_USERID')"
#   * "- Trường 2 (ví dụ: `quantity`): (Bắt buộc/Không, Kiểu dữ liệu, Ràng buộc, ví dụ: 'phải là số nguyên > 0', mã lỗi trả về nếu có: 'E_INVALID_QUANTITY')"
#   * "- (Tiếp tục cho tất cả các trường trong request...)"
#   * "- Tôi cũng sẽ tìm kiếm các tương quan validation giữa các trường (ví dụ: `startDate` phải trước `endDate`, hoặc một trường không được trùng với trường khác), và các mã lỗi liên quan nếu có."

# * **Phân tích Business Logic (BL) và Dữ liệu:**
#   "Bây giờ, tôi sẽ phân tích các quy tắc nghiệp vụ trong `QUY TẮC HÀNH VI` (thường trả về 200, 422, 404, hoặc các mã nghiệp vụ khác) và liên kết chúng trực tiếp với `DỮ LIỆU CHO KIỂM THỬ` (nếu có):"
#   * **Quy tắc X (Happy Path):** "Cần những điều kiện gì để thành công? (ví dụ: 'Người dùng A có đủ quyền, Sản phẩm B còn hàng'). Tôi sẽ tìm dữ liệu tương ứng trong `DỮ LIỆU CHO KIỂM THỬ` (ví dụ: 'Dùng user A (ACTIVE)', 'Dùng product B (Stock=100)')."
#   * **Quy tắc Y (Business Error 1):** "Cần gì để kích hoạt lỗi này? (ví dụ: 'Thực hiện hành động trên một tài nguyên đã bị khóa'). Tôi sẽ tìm dữ liệu tương ứng (ví dụ: 'Dùng resource C (LOCKED)')."
#   * **Quy tắc Z (Business Error 2):** "Cần gì để kích hoạt lỗi này? (ví dụ: 'Giá trị vượt quá giới hạn nghiệp vụ'). Tôi sẽ tìm dữ liệu tương ứng (ví dụ: 'Dùng tài khoản Y (Limit=100)' và thử 'amount=200')."
#   * "(Tiếp tục cho tất cả các Quy tắc Hành vi...)"

# * **Lập chiến lược Test Case:**
#   "Tôi sẽ tạo test case cho từng mục trong BV và BL đã phân tích.
#   Tôi sẽ LIỆT KÊ DANH SÁCH các chiến lược kiểm thử như sau:"
#   * "- Với BV, áp dụng các kỹ thuật như phân vùng tương đương và phân tích giá trị biên (ví dụ: kiểm tra giá trị = 0, giá trị âm, giá trị lớn nhất, giá trị biên)."
#   * "- Kiểm tra các trường hợp không hợp lệ (ví dụ: gửi một giá trị enum không được hỗ trợ)."
#   * "- Kiểm tra trường hợp thiếu dữ liệu (ví dụ: một trường string dài hơn giới hạn), sử dụng `KEYWORDS ĐẶC BIỆT` như `CHARS(n)`."
#   * "- Với BL, tạo test case cho từng quy tắc nghiệp vụ, bao gồm cả các trường hợp thành công (happy path) và các trường hợp lỗi nghiệp vụ (business error)."

# ## 2. YÊU CẦU NGHIÊM NGẶT

# * **Ngôn ngữ** của **CHAIN OF THOUGHT** phải **giống với ngôn ngữ của bài toán**.
# * Sau reasoning phải kết thúc bằng dòng **"Stop Thinking..."**.
# * Ngay sau dòng "Stop Thinking...", tạo một đối tượng JSON duy nhất (một mảng các test case), được đặt trong block code dạng json theo `CẤU TRÚC ĐẦU RA` đã được định nghĩa.
# * Khối JSON PHẢI được bọc trong block code với ngôn ngữ là `json`.
# * Output JSON PHẢI ở dạng "pretty" (được format dễ đọc, có thụt lề).
# * KHÔNG GIẢI THÍCH BẤT CỨ ĐIỀU GÌ sau khối JSON.
# * Tuân thủ nghiêm ngặt cấu trúc đầu ra sau:
# * **Lưu ý:** Mỗi nhóm test case `basic_validation` và `business_logic` phải có bộ `test_case_id` riêng biệt, mỗi bộ bắt đầu từ 1.
# * **Nguyên tắc:** Mọi reasoning (luồng suy nghĩ) phải cô đọng nhất có thể, tránh dài dòng.
# * **Yêu cầu bổ sung:** Khi sinh test case cho phần `basic_validation`, phải cover đầy đủ tất cả các loại kiểm thử: thiếu trường (`ABSENT`), giá trị `NULL`, giá trị rỗng (`N/A`), giá trị biên, kiểu dữ liệu sai, giá trị không hợp lệ, giá trị vượt giới hạn, v.v. **Đối với mỗi loại kiểm thử, PHẢI kiểm tra cả trường hợp hợp lệ và không hợp lệ (ví dụ: với giá trị biên thì phải test cả hai biên: valid và invalid).**
# * Khi kiểm thử giá trị biên hoặc số lượng ký tự, **BẮT BUỘC** sử dụng các keyword đặc biệt (`CHARS(n)`, `NUMS(n)`, ...) thay vì dữ liệu cụ thể.
# * Tất cả các test case sinh ra đều **PHẢI** dựa trên QUY TẮC HÀNH VI. Nếu QUY TẮC HÀNH VI không đề cập đến một quy tắc hoặc trường hợp, KHÔNG được tự ý bịa đặt hoặc giả định quy tắc/nghiệp vụ.

# ## 3. CẤU TRÚC ĐẦU RA

# Sau reasoning, kết thúc bằng dòng **"Stop Thinking..."** rồi đến block code json:

# ```json
# {output_json_example}
# ```

# **Giải thích các trường:**

# * `request_body`: Đối tượng đại diện cho payload mẫu ban đầu dựa trên `DỮ LIỆU CHO KIỂM THỬ`, từ đó bạn sẽ tạo các biến thể trong `request_mapping` cho từng test case.
# * `test_case_id`: ID số nguyên, **bắt đầu từ 1 cho mỗi nhóm** test case (`basic_validation` và `business_logic`), tăng dần trong từng nhóm.
# * `basic_validation`: bộ testcases Kiểm tra ràng buộc cơ bản của *từng trường* như định dạng, độ dài, kiểu dữ liệu, giá trị hợp lệ.
# * `business_logic`: bộ testcases Kiểm tra các quy tắc nghiệp vụ, sự tương quan giữa các trường.
# * `test_case`: Tiêu đề mô tả ngắn gọn kịch bản kiểm thử, luôn tuân thủ cấu trúc:
#   * `"<field_name> with <condition/value> should <expected_result>"`
#   * Ví dụ: `"user.age with NULL should return statuscode 400"` hoặc `"transaction.amount with negative value should return error E_INVALID_AMOUNT"`
# * `request_mapping`: Đối tượng đại diện cho payload. Sử dụng các keywords đặc biệt nếu cần. Chỉ cần thay đổi các trường liên quan đến kịch bản kiểm thử, hệ thống sẽ tự động sử dụng giá trị từ `request_body` cho các trường không được đề cập.
# * `response_mapping`: (Tùy chọn) Đối tượng đại diện cho các trường cần xác thực trong body của response. Ví dụ: `{response_mapping_example}`.
# * `expected_output`: Đối tượng chứa kết quả mong đợi, bao gồm `statuscode` và `response_mapping`.

# ## 4. KEYWORDS ĐẶC BIỆT CHO TRƯỜNG `request_mapping`

# Khi tạo payload trong `request_mapping`, bạn PHẢI sử dụng các keywords sau khi thích hợp để đại diện cho các giá trị đặc biệt. KHÔNG được tự ý sinh dữ liệu cụ thể, chỉ sử dụng đúng các keyword này. Các keywords này là các chuỗi ký tự và phải được đặt trong dấu ngoặc kép.

# * `"N/A"`: Gán chuỗi rỗng (empty string `""`).
# * `"NULL"`: Gán giá trị `null` cho trường.
# * `"ABSENT"`: Xóa hoàn toàn trường này khỏi payload.
# * `"CHARS(n)"`: Một chuỗi ký tự bất kỳ có độ dài `n`.
# * `"NUMS(n)"`: Một chuỗi số có độ dài `n`.
# * `"ALPHANUMS(n)"`: Một chuỗi chữ và số có độ dài `n`.
# * `"EMAIL(n)"`: Một địa chỉ email hợp lệ có độ dài `n`.

# ---

# BÀI TOÁN:
# {document}

# ### Output:

# '''

response_mapping_example = '''{"user_id": "<some_value>"}` hoặc `{"error_code": "INVALID_INPUT"}'''

output_json_example = '''
{
  "request_body": {
    "<field_name>": "<value>",
    ...
  },
  "testcases": {
    "basic_validation": [
      {
        "test_case_id": <integer>,
        "test_case": "<test_case_title>",
        "request_mapping": {
          "<field_name>": "<value>",
          ...
        },
        "expected_output": {
          "statuscode": <integer>,
          "response_mapping": {
            "field_name": "<expected_value>",
            ...
          }
        }
      }
      ...
    ],
    "business_logic": [
      {
        "test_case_id": <integer>,
        "test_case": "<test_case_title>",
        "request_mapping": {
          "<field_name>": "<value>",
          ...
        },
        "expected_output": {
          "statuscode": <integer>,
          "response_mapping": {
            "field_name": "<expected_value>",
            ...
          }
        }
      }
      ...
    ]
  }
}
'''

In [ ]:
class APIKeys():
    def __init__(self, api_keys: list[str]):
        self.api_keys = api_keys
        self.index = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.index < len(self.api_keys):
            key = self.api_keys[self.index]
            self.index += 1
            return key

        raise StopIteration

    def __get_item__(self, index):
        return self.api_keys[index]

    def __len__(self):
        return len(self.api_keys)

    def __contains__(self, key):
        return key in self.api_keys

    def get_next_key(self):
        self.index = (self.index + 1) % len(self.api_keys)
        return self.api_keys[self.index]

In [ ]:
api_keys = APIKeys(GOOGLE_API_KEYS)

current_key = api_keys.get_next_key()
print(current_key[-6:-1])

-771O


In [ ]:
# !gdown 1GN0S7pHycuSkhDhYhCD_W2TydNsNiryI

Downloading...
From: https://drive.google.com/uc?id=1GN0S7pHycuSkhDhYhCD_W2TydNsNiryI
To: /content/API_testcase_v2.json
100% 486k/486k [00:00<00:00, 108MB/s]


In [ ]:
# all_combinations_df.to_dicts()

In [ ]:
data_df = pl.read_csv("/content/final_data_stage1.csv")
# data_df = data_df.rename({"response": "input"})
data_df

id,domain,api_functionality,api_documentation_structure,api_detail_structure,behaviour_rule_structure,data_test_structure,data_test_full_fill,biz_code_type,response_structure,api_documentation_num,request_body_field_num,behaviour_rule_num,data_test_num,language,input
i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str
1,"""Ngân hàng""","""Chuyển tiền""","""plain text""","""table""","""list""","""list""","""full""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",3,3,5,4,"""Vietnamese""","""* Domain: **Ngân hàng** * …"
2,"""Ngân hàng""","""Chuyển tiền""","""table""","""plain text""","""list""","""table""","""partial""","""number (1001)""","""Data nested within 'data'""",3,3,5,3,"""Vietnamese""","""* Domain: **Ngân hàng** * …"
3,"""Ngân hàng""","""Chuyển tiền""","""plain text""","""list""","""table""","""table""","""empty""","""string (E_INVALID_AMOUNT)""","""Flat data""",5,5,3,5,"""Vietnamese""","""Domain: **Ngân hàng** Chức năn…"
4,"""Ngân hàng""","""Chuyển tiền""","""list""","""list""","""list""","""table""","""full""","""number (1001)""","""Flat data""",3,3,3,3,"""Vietnamese""","""* Domain: **Ngân hàng** * …"
5,"""Ngân hàng""","""Chuyển tiền""","""table""","""plain text""","""table""","""list""","""empty""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",5,5,3,5,"""Vietnamese""","""Domain: **Ngân hàng** Chức năn…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1396,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""plain text""","""plain text""","""table""","""table""","""empty""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",5,5,5,3,"""English""","""Domain: **Retail** Function: *…"
1397,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""table""","""plain text""","""list""","""list""","""empty""","""string (E_INVALID_AMOUNT)""","""Flat data""",4,3,3,5,"""English""","""Domain: Retail Function: View …"
1398,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""table""","""plain text""","""table""","""list""","""partial""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",3,3,3,4,"""English""","""Domain: **Retail** Function: *…"


In [ ]:
vn_data_df = data_df.filter(pl.col("language") == "Vietnamese")
en_data_df = data_df.filter(pl.col("language") == "English")

In [ ]:
# from openai import OpenAI
from itertools import batched

def get_instruction(_sys_instruct: str,
                 input_document: str):
    # client = OpenAI(
    #     base_url="https://ai.megallm.io/v1",
    #     api_key=api_key
    # )

    __sys_instruct = _sys_instruct.format(
        document=input_document,
        response_mapping_example=response_mapping_example,
        output_json_example=output_json_example
    )

    # response = client.chat.completions.create(
    #     model="gemini-2.5-pro",
    #     temperature=0,
    #     messages=[
    #         {"role": "user", "content": _sys_instruct}
    #     ]
    # )

    # response = client.models.generate_content(
    #     model=model_name,
    #     config=types.GenerateContentConfig(
    #         system_instruction=_sys_instruct,
    #         # Moved max_output_tokens and temperature inside config
    #         temperature=0
    #     ),
    #     contents=['']
    # )
    # # return response.choices[0].message.content
    # return response.text

    return __sys_instruct

def create_df_with_instruction(df: pl.DataFrame):
    instruction_df = pl.DataFrame({
        "instruction": [get_instruction(sys_instruct, row_data) for row_data in df.rows(named=True)]
    })
    return pl.concat([df, instruction_df], how="horizontal")

def get_inline_request_batches(df: pl.DataFrame, batch_size: int = 100):
    df = create_df_with_instruction(df)

    preprocessed_requests = [
        {
            'contents': [{
                'parts': [{'text': instruction}],
                'role': 'user'
            }],
            'config': {'temperature': 0.1}
        } for instruction in df["instruction"].to_list()
    ]

    return list(batched(preprocessed_requests, batch_size))

In [ ]:
# from pprint import pprint

# for batch in get_inline_request_batches(data_df, 2):
#     pprint(batch)
#     break

In [ ]:
from google.genai.types import BatchJob

def send_request(index, batch, client: Client) -> str|None:
    inline_batch_job = client.batches.create(
        model=f"models/{model_name}",
        src=batch,
        config={
            'display_name': f"inlined-requests-{i}",
        },
    )
    return inline_batch_job.name

def check_batch_status(batch_job_name: str, client: Client):
    batch_job = client.batches.get(name=batch_job_name)
    return batch_job.state.name

def get_inline_responses(batch_job_name: str, client: Client):
    inline_responses = []
    batch_job = client.batches.get(name=batch_job_name)
    if batch_job.dest and batch_job.dest.inlined_responses:
        for i, inline_response in enumerate(batch_job.dest.inlined_responses):
            if inline_response.response:
                # Accessing response, structure may vary.
                try:
                    inline_responses.append(inline_response.response.text)
                except AttributeError:
                    print(inline_response.response) # Fallback
                    inline_responses.append(None)
            elif inline_response.error:
                print(f"Error: {inline_response.error}")
                inline_responses.append(None)
    else:
        print("No results found (neither file nor inline).")
    return inline_responses

In [ ]:
# --- Load final_data (CSV) ---
final_data_path = "/content/final_data_stage2.csv"
if os.path.exists(final_data_path):
    try:
        final_data_df = pl.read_csv(final_data_path)
        final_data = final_data_df.to_dicts() # Keep as list of dicts for easy appending
        processed_id = final_data_df.get_column('id').to_list()
        print(f"Loaded {len(final_data)} samples from {final_data_path}")
    except Exception as e:
        print(f"Error loading final_data.csv: {e}. Initializing empty list.")
        final_data = []
        processed_id= []
else:
    final_data = []
    processed_id = []
    print(f"No sample found in {final_data_path}")

Loaded 700 samples from /content/final_data_stage2.csv


In [ ]:
unprocessed_df = en_data_df.filter(~pl.col("id").is_in(processed_id))
unprocessed_df

id,domain,api_functionality,api_documentation_structure,api_detail_structure,behaviour_rule_structure,data_test_structure,data_test_full_fill,biz_code_type,response_structure,api_documentation_num,request_body_field_num,behaviour_rule_num,data_test_num,language,input
i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str
701,"""Ngân hàng""","""Chuyển tiền""","""list""","""list""","""table""","""table""","""partial""","""number (1001)""","""Data nested within 'data'""",4,3,4,5,"""English""","""Domain: Banking Function: Mone…"
702,"""Ngân hàng""","""Chuyển tiền""","""list""","""plain text""","""table""","""table""","""empty""","""number (1001)""","""Flat data""",4,4,4,4,"""English""","""Domain: Banking Function: Mone…"
703,"""Ngân hàng""","""Chuyển tiền""","""plain text""","""table""","""table""","""table""","""empty""","""string (E_INVALID_AMOUNT)""","""Flat data""",3,5,3,4,"""English""","""Domain: **Banking** Function: …"
704,"""Ngân hàng""","""Chuyển tiền""","""table""","""list""","""table""","""list""","""empty""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",3,5,4,5,"""English""","""Domain: **Banking** Function: …"
705,"""Ngân hàng""","""Chuyển tiền""","""plain text""","""list""","""list""","""table""","""full""","""number (1001)""","""Data nested within 'data'""",5,3,4,3,"""English""","""Domain: **Banking** Function: …"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1396,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""plain text""","""plain text""","""table""","""table""","""empty""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",5,5,5,3,"""English""","""Domain: **Retail** Function: *…"
1397,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""table""","""plain text""","""list""","""list""","""empty""","""string (E_INVALID_AMOUNT)""","""Flat data""",4,3,3,5,"""English""","""Domain: Retail Function: View …"
1398,"""Bán lẻ""","""Xem lịch sử đơn hàng""","""table""","""plain text""","""table""","""list""","""partial""","""string (E_INVALID_AMOUNT)""","""Data nested within 'data'""",3,3,3,4,"""English""","""Domain: **Retail** Function: *…"


In [ ]:
# processed_df = unprocessed_df.clear()
# processed_df = processed_df.with_columns([
#     pl.col("input").alias("output")
# ])
# processed_df

In [ ]:
from google.colab import files

batch_size = 100
batches = get_inline_request_batches(unprocessed_df, batch_size)

sleep_time = 120

client = Client(api_key=current_key)

try:
    for i, batch in tqdm(enumerate(batches), total=len(batches)):
        batch_job_name = send_request(i, list(batch), client)
        batch_job = client.batches.get(name=batch_job_name)

        completed_states = set([
            'JOB_STATE_SUCCEEDED',
            'JOB_STATE_FAILED',
            'JOB_STATE_CANCELLED',
            'JOB_STATE_EXPIRED',
        ])

        print(f"Polling status for job: {batch_job_name}")
        while check_batch_status(batch_job_name, client) not in completed_states:
            print(f"Current state: {check_batch_status(batch_job_name, client)} - Waiting for {sleep_time} seconds for the next polling.")
            time.sleep(sleep_time)

        print(f"Job finished with state: {check_batch_status(batch_job_name, client)}")
        if check_batch_status(batch_job_name, client) == 'JOB_STATE_FAILED':
            print(f"Error: {batch_job.error}")


        if check_batch_status(batch_job_name, client) == 'JOB_STATE_SUCCEEDED':
            responses = get_inline_responses(batch_job_name, client)
            responses_df = pl.DataFrame(
                {"output": responses}
            )
            responses_df = pl.concat([unprocessed_df[(i*batch_size):(i*batch_size+len(batch))], responses_df], how='horizontal')
            # processed_df = pl.concat([processed_df, responses_df], how='vertical')
            responses_df = responses_df.select(final_data_df.columns)
            final_data_df = pl.concat([final_data_df, responses_df], how='vertical')
            final_data_df.write_csv(final_data_path)
        else:
            print(f"Job did not succeed. Final state: {batch_job.state.name}")
            if batch_job.error:
                print(f"Error: {batch_job.error}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    final_data_df.write_csv(final_data_path)
    files.download(final_data_path)



  0%|          | 0/7 [00:00<?, ?it/s]

Polling status for job: batches/rvbaosmlzh1gukfxauy18m0sk8e8xrje8fxf
Current state: JOB_STATE_PENDING - Waiting for 120 seconds for the next polling.
Current state: JOB_STATE_RUNNING - Waiting for 120 seconds for the next polling.
Job finished with state: JOB_STATE_SUCCEEDED
Polling status for job: batches/mjfqmord9alo2utnnjrpssh6v8vh655d2qxh
Current state: JOB_STATE_PENDING - Waiting for 120 seconds for the next polling.
Current state: JOB_STATE_RUNNING - Waiting for 120 seconds for the next polling.
Job finished with state: JOB_STATE_SUCCEEDED
Polling status for job: batches/nhhsxaobmt24qu06mkx6uq41xt4x60e87w1b
Current state: JOB_STATE_PENDING - Waiting for 120 seconds for the next polling.
Current state: JOB_STATE_RUNNING - Waiting for 120 seconds for the next polling.
Job finished with state: JOB_STATE_SUCCEEDED
Polling status for job: batches/y1btkvx85hgmtc3rkni4s9je86jm7u093yr4
Current state: JOB_STATE_PENDING - Waiting for 120 seconds for the next polling.
Current state: JOB_STA

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download("/content/final_data_stage2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>